In [ ]:
from fastai.text.all import *
from fastcore.all import *

In [ ]:
path = Path()

In [ ]:
class SpacyTokenizerBig():
# Измененный класс токенайзера для поддержки словаря большого размера
    def __init__(self, lang='en', special_toks=None, buf_sz=5000):
        import spacy
        from spacy.symbols import ORTH
        self.special_toks = ifnone(special_toks, defaults.text_spec_tok)
        nlp = spacy.blank(lang)
        nlp.max_length = 10**10
        for w in self.special_toks: nlp.tokenizer.add_special_case(w, [{ORTH: w}])
        self.pipe,self.buf_sz = nlp.pipe,buf_sz

    def __call__(self, items):
        return (L(doc).attrgot('text') for doc in self.pipe(map(str,items), batch_size=self.buf_sz))

In [ ]:
tok = SpacyTokenizerBig()

In [ ]:
data_path = (path/'HarryPotter_txt')

In [ ]:
data_path = (path/'StephenKing_txt')

In [ ]:
text_block = DataBlock(
    blocks=TextBlock.from_folder(data_path, is_lm=True, tok=tok),
    get_items = get_text_files,
    splitter=RandomSplitter(0.1)
)

In [ ]:
dls = text_block.dataloaders(data_path, path=data_path, bs=256, seq_len=64)

In [ ]:
dls.show_batch(max_n=6)

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save('writer_HarPot_64_10epoch')

In [ ]:
learn.save('writer_StKng_64_10epoch')

In [ ]:
learn.export(path/'WriterHP_model.pkl')

In [ ]:
learn.export(path/'WriterStK_model.pkl')

In [ ]:
TEXT = "It was a bright day"
N_WORDS = 200
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=1) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))